In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import BooleanType, ArrayType, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, NGram
from functools import reduce

import pandas as pd
import re
import nltk

In [2]:
spark = (SparkSession.builder \
    .appName("SPARK TRAB - QUESTION 1 D").getOrCreate())

In [3]:
df_orig = spark.read.option("header", "false").option("delimiter", "\t").csv("./data/debate-tweets.tsv")

## Objetivo

In [4]:
df = df_orig.select("_c0", "_c1")

df = df.withColumnRenamed("_c0", "id") \
                     .withColumnRenamed("_c1", "content")

df.show()

+------------------+--------------------+
|                id|             content|
+------------------+--------------------+
|522394422710136832|@anacddd verdade,...|
|522394422806581248|              Que ñ*|
|522394422731100160| Vou quebrar a Bruna|
|522394422810783745|agora vou p segun...|
|522394423137943553|Me sinto tão bem ...|
|522394423188271104|Eu estou aqui, de...|
|522394423238606848|Quando vai embora...|
|522394423528022016|@paynecaralhudo k...|
|522394423632875521|Conceição da Barr...|
|522394424010362881| @Maniavato te amo ♥|
|522394424048091138|Alg me curtindo rs ♡|
|522394424010358784|@MiiluAA No, porq...|
|522394423741906944|#EMABiggestFansJu...|
|522394424568213505|@raizabatista dev...|
|522394424920506368|Me senti ate d fe...|
|522394424811458560|qual o sentido de...|
|522394425029574656|I'm at Lava Rápid...|
|522394425121841153|Fica comentando m...|
|522394425461579777|"odeio que me man...|
|522394425960701952|CAMAMTEBABILONFRA...|
+------------------+--------------

In [5]:
def extract_dilma(text):
    if re.search(r'\bDilma\b', text, re.IGNORECASE):
        return True
    else:
        return False
    
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', ' ', text)

In [6]:
extract_dilma_udf = udf(extract_dilma, BooleanType())

df = df.withColumn("hasDilma", extract_dilma_udf(df["content"]))

df_dilma = df.filter(col("hasDilma")).select("id", "content")

display(df_dilma.head(20))

[Row(id='522394534773538816', content='"""@marinaecologia: Dilma não arruma nem os dentes, imagina o país."""'),
 Row(id='522394590671020032', content='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma'),
 Row(id='522394744463577088', content='O gigante acordou tanto que vão votar na Dilma de novo 😂😂😂'),
 Row(id='522394798914015233', content='@KennedyAncar @Drimone @jornaldaREALMENTE DILMA CAGUEJOU,ESTAVA NERVOSA,SEM PREPARO,MUITO REPETITIVA VOLTA MUITO NO PASSADO,AÉCIO FOI MELHOR'),
 Row(id='522394887472562176', content='Quem vota na Dilma merece apanhar por que pqp hein'),
 Row(id='522395038056058880', content='OLHA DILMA MINHA FILHA ENTENDA FILHINHA VOCÊ JÁ ERA NÃO RESISTE TOMA MEU LENÇO: #45Confirma ! !!! !!! !!!'),
 Row(id='522395096290185216', content='@BLOGdoPEPE mas Noblat reconhece superioridade de Dilma.'),
 Row(id='522395241828323328', content='"Dilma, viado, continua que tá bonito! ""Não, não me abandone. Não me desespere, porque eu não posso ficar sem 

In [7]:
remove_punctuation_udf = udf(remove_punctuation, StringType())

df_dilma = df_dilma.withColumn("content_clean", remove_punctuation_udf(col("content")))

display(df_dilma.head(20))

[Row(id='522394534773538816', content='"""@marinaecologia: Dilma não arruma nem os dentes, imagina o país."""', content_clean='    marinaecologia  Dilma não arruma nem os dentes  imagina o país    '),
 Row(id='522394590671020032', content='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma', content_clean='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma'),
 Row(id='522394744463577088', content='O gigante acordou tanto que vão votar na Dilma de novo 😂😂😂', content_clean='O gigante acordou tanto que vão votar na Dilma de novo    '),
 Row(id='522394798914015233', content='@KennedyAncar @Drimone @jornaldaREALMENTE DILMA CAGUEJOU,ESTAVA NERVOSA,SEM PREPARO,MUITO REPETITIVA VOLTA MUITO NO PASSADO,AÉCIO FOI MELHOR', content_clean=' KennedyAncar  Drimone  jornaldaREALMENTE DILMA CAGUEJOU ESTAVA NERVOSA SEM PREPARO MUITO REPETITIVA VOLTA MUITO NO PASSADO AÉCIO FOI MELHOR'),
 Row(id='522394887472562176', content='Quem vota na Dilma merece apanhar por que

In [8]:
tokenizer = Tokenizer(inputCol="content_clean", outputCol="words")
df_dilma = tokenizer.transform(df_dilma)

display(df_dilma.head(20))

[Row(id='522394534773538816', content='"""@marinaecologia: Dilma não arruma nem os dentes, imagina o país."""', content_clean='    marinaecologia  Dilma não arruma nem os dentes  imagina o país    ', words=['', '', '', '', 'marinaecologia', '', 'dilma', 'não', 'arruma', 'nem', 'os', 'dentes', '', 'imagina', 'o', 'país']),
 Row(id='522394590671020032', content='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma', content_clean='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma', words=['pensando', 'seriamente', 'em', 'chamar', 'o', 'manicômio', 'pra', 'quem', 'quer', 'votar', 'na', 'dilma']),
 Row(id='522394744463577088', content='O gigante acordou tanto que vão votar na Dilma de novo 😂😂😂', content_clean='O gigante acordou tanto que vão votar na Dilma de novo    ', words=['o', 'gigante', 'acordou', 'tanto', 'que', 'vão', 'votar', 'na', 'dilma', 'de', 'novo']),
 Row(id='522394798914015233', content='@KennedyAncar @Drimone @jornaldaREALMENTE DILMA 

In [9]:
def remove_tokens(tokens):
    return [token for token in tokens if len(token) >= 3]

In [10]:
stopwordList = nltk.corpus.stopwords.words('portuguese')

remover = StopWordsRemover(inputCol="words", outputCol="filtered", stopWords = stopwordList)
df_dilma = remover.transform(df_dilma)

remove_tokens_udf = udf(remove_tokens, ArrayType(StringType()))

df_dilma = df_dilma.withColumn("filtered", remove_tokens_udf(col("filtered")))

display(df_dilma.head(20))

[Row(id='522394534773538816', content='"""@marinaecologia: Dilma não arruma nem os dentes, imagina o país."""', content_clean='    marinaecologia  Dilma não arruma nem os dentes  imagina o país    ', words=['', '', '', '', 'marinaecologia', '', 'dilma', 'não', 'arruma', 'nem', 'os', 'dentes', '', 'imagina', 'o', 'país'], filtered=['marinaecologia', 'dilma', 'arruma', 'dentes', 'imagina', 'país']),
 Row(id='522394590671020032', content='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma', content_clean='Pensando seriamente em chamar o manicômio pra quem quer votar na Dilma', words=['pensando', 'seriamente', 'em', 'chamar', 'o', 'manicômio', 'pra', 'quem', 'quer', 'votar', 'na', 'dilma'], filtered=['pensando', 'seriamente', 'chamar', 'manicômio', 'pra', 'quer', 'votar', 'dilma']),
 Row(id='522394744463577088', content='O gigante acordou tanto que vão votar na Dilma de novo 😂😂😂', content_clean='O gigante acordou tanto que vão votar na Dilma de novo    ', words=['o', 'g

In [11]:
def generate_ngrams(wordsData, ngram_range):
    ngrams_data = []
    for n in range(ngram_range[0], ngram_range[1] + 1):
        ngram = NGram(n=n, inputCol="filtered", outputCol="ngram")
        ngram_data = ngram.transform(wordsData)
        ngrams_data.append(ngram_data)
    return reduce(lambda df1, df2: df1.union(df2), ngrams_data)

In [12]:
# ngram_range = (3, 5)
# df_dilma = generate_ngrams(df_dilma, ngram_range)

ngram = NGram(n = 3, inputCol = "filtered", outputCol="ngram")
df_dilma = ngram.transform(df_dilma)

df_dilma.show() # OK

+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                id|             content|       content_clean|               words|            filtered|               ngram|
+------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|522394534773538816|"""@marinaecologi...|    marinaecologi...|[, , , , marinaec...|[marinaecologia, ...|[marinaecologia d...|
|522394590671020032|Pensando seriamen...|Pensando seriamen...|[pensando, seriam...|[pensando, seriam...|[pensando seriame...|
|522394744463577088|O gigante acordou...|O gigante acordou...|[o, gigante, acor...|[gigante, acordou...|[gigante acordou ...|
|522394798914015233|@KennedyAncar @Dr...| KennedyAncar  Dr...|[, kennedyancar, ...|[kennedyancar, dr...|[kennedyancar dri...|
|522394887472562176|Quem vota na Dilm...|Quem vota na Dilm...|[quem, vota, na, ...|[vota, dilma, mer...|[vota dilma me

In [13]:
ngrams = df_dilma.select("ngram")
ngrams = ngrams.withColumn("sentences", explode("ngram")).select("sentences")
top_results = ngrams.groupBy("sentences").count().orderBy(col("count").desc()).limit(100)

In [14]:
#important_sentences_df.show()

In [15]:
dataframe_pd = top_results.toPandas()

dataframe_pd.to_csv('./outcome/Q1/result_sentences_dilma.csv', index=False)

In [16]:
spark.stop()

In [17]:
# REDUZIR NUMERO DE FEATURES
# CHECAR MOSTRAR O RESUTLADO SEM TER A FINAL_FEATURES E PRONTO

In [18]:
# VOU AGORA PEGAR EXPLODIR NGRAM E FAZER AS MAIS FREQUENTES
# MELHORAR ESSE COUNT PARA ESSE NÃO SER TÃO SIMPLES E TENTAR FAZER ALGO COMO SOMA DAS SEMELHANÇAS